<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/binary_nc_converter/single_level_incr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## To read (binary formatted) NCODA/RTOFS DA increment file and write it out to a netcdf formatted file.

Inspired by [Dmitry Dukhovsky's increment file reader.](https://github.com/DmitryDukhovskoy/RTOFS_utilities/blob/master-hera/MyPython/ncoda_utils/mod_read_ncoda.py)

In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_ncoda_increment_2d(data_path, fName):

  vName =fName.split('_')[0]
  im, jm = [int(fName.split('_')[2][2:6]), int(fName.split('_')[2][7:11])]
  fDate = fName.split('_')[3]
  fDate = fDate[0:4] + '-' + fDate[4:6] + '-' + fDate[6:8]# + ':' + fDate[8:10] # Always at 00 UTC
  fTime = np.array([str(fDate)], dtype='datetime64')

  print(f'\nReading RTOFS DA {vName} increment on\n{fTime} with [x,y] dim = {im,jm}.\n')

  f = open(data_path + fName, 'rb')
  vals = []
  f.seek(0)
  dummy = np.fromfile(f, dtype='>f',count=jm*im) # read 2d file (1 layer)
  #dummy = dummy.reshape((jm,im))
  #vals = np.copy(dummy)
  vals = dummy.reshape((jm,im))
  f.close()

  return vName, fTime, vals

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

dPath = '/content/drive/MyDrive/datasets/tmp/'

## Need:
- Grid (Longitude, Latitude).
- Bathymetry information.

We'll use them to create land-sea mask

## Get them from hycom.org

In [ ]:
!wget https://data.hycom.org/datasets/GLBb0.08/expt_93.0/topo/depth_GLBb0.08_09m11.nc

In [ ]:
ds_topo = xr.open_dataset('depth_GLBb0.08_09m11.nc', decode_times=False)

In [ ]:
ds_topo

## Create a land-sea mask from bathymetry:
- land: `0`.
- sea: `1`.

In [ ]:
ls_mask = ds_topo.copy(deep=True)
ls_mask['depth'] = xr.where(np.isnan(ls_mask.depth), 0, 1)
ls_mask = ls_mask.rename({'depth':'mask'}) # rename

In [ ]:
ls_mask

## Visually verify to make sure we got what we wanted

In [ ]:
ls_mask.mask.plot(vmin=0, vmax=1, cmap='Set1_r')

In [ ]:
# Read an increment file
fName = 'icecov_sfc_1o4500x3298_2024123100_0000_analinc'
vName, incDate, ice_cov = read_ncoda_increment_2d(dPath, fName)

# This increment in ice coverage is in percentage (why??!!)
ice_cov = ice_cov / 100 # convert it to [-1, 1]

## Create a dataset with ice coverage (that was read in from binary file)

In [ ]:
ds_ice_cov = xr.open_dataset('depth_GLBb0.08_09m11.nc', decode_times=False)
ds_ice_cov['ice_cov'] = (('Y', 'X'), ice_cov)

In [ ]:
# Make sure concentration over land = 0.
# Land values will be made to nan anyway, so this is done only for sanity sake!
ds_ice_cov['ice_cov'] = ds_ice_cov.ice_cov * ls_mask.mask.squeeze()

In [ ]:
ds_ice_cov.ice_cov.plot(vmin=-1, vmax=1, cmap='Spectral_r')

## Ice concentration:
- Has values ranging from 0- 1 (as above).
- We can't tell (visually) where land/sea is!

Apply the land sea mask created above

In [ ]:
ds_ice_cov['ice_cov'] = ds_ice_cov.ice_cov.where(ls_mask.mask == 1, np.nan)

In [ ]:
ds_ice_cov.ice_cov.plot(vmin=-1, vmax=1, cmap='Spectral_r')

## Save the ice coverage to a netcdf file:
- Only write ice coverage; drop bathymetry.
- Fix attributes.


In [ ]:
ds_ice_cov = ds_ice_cov.drop_vars(['depth', 'Date']) # drop bathymetry

ds_ice_cov=ds_ice_cov.rename({'MT': 'time'}) # rename MT to time
ds_ice_cov['time'] = incDate # add time value

# fix attributes
ds_ice_cov = ds_ice_cov.drop_attrs(deep=True)
ds_ice_cov.ice_cov.attrs['units'] = '1'
ds_ice_cov.ice_cov.attrs['standard_name'] = vName
ds_ice_cov.ice_cov.attrs['description'] = 'Increment in sea ice concentration'
ds_ice_cov.attrs['source'] = 'NCEP RTOFS v2.5'

# save file
ds_ice_cov.to_netcdf('ice_cov.nc')

## Read back in to verify

In [ ]:
xx = xr.open_dataset('ice_cov.nc')
xx

In [ ]:
xx.ice_cov.plot(vmin=-1, vmax=1, cmap='Spectral_r')